# Shimano E-Tube Reversing

- Create virtual python environment in repo root and switch to it
- Run `pip install --upgrade plotly pandas nbformat` in your venv

In [5]:
# Init
import plotly.express as px
import pandas as pd
import os
import math

# Params
threshold = 0.2         # Absolute voltage above which the signal is assumed HIGH/LOW
frequency = 1000000     # Carrier frequency in Hz. 1MHz for E-Tube


# Read in and attempt to translate a single burst of transmission

Point `relPath` to a file in `data/scopes` to have translated

In [52]:
relPath = '2024-01-01_ButtonDownSingleSegment/segment2.csv'

# Find data folder
notebookFolder      = (os.path.abspath(''))
repoRoot            = os.path.dirname(notebookFolder)
intermediatePath    = os.path.join(repoRoot, 'data')
intermediatePath    = os.path.join(intermediatePath, 'scopes')
sourceFile          = os.path.join(intermediatePath, relPath)

# Read in button down csv
df = pd.read_csv(sourceFile)

# Drop garbage data at beginning
df = df.drop(index=[0,1])

# Convert scientific notation to float
df['time']  = df['x0000']
df['voltage'] = df['y0000']
df = df.drop(columns=['x0000', 'y0000'])

# Add 'latched' square-ish wave
df['voltageLatched'] = df['voltage'].apply(lambda x: df['voltage'].max() if x > threshold else (df['voltage'].min() if x < -threshold else 0))

# Detect peak at start by iterating over dataframe
minimumPeak = df['voltage'].max() * 0.8
previousPeak = 0
peakTime = 0
for i in df.index:
    # Raise peak found value continuously
    if df['voltage'][i] > previousPeak:
        previousPeak = df['voltage'][i]
        peakTime = df['time'][i]
    # If we are on a falling edge past a peak of sufficient height, assume the previous peak was a confident one
    elif previousPeak > minimumPeak and previousPeak > df['voltage'][i]:
        break

print('Peak found at: ' + str(peakTime) + ' with a height of: ' + str(previousPeak))

# Create 'markers' with configured frequency starting at first peak found
startTime   = peakTime
endTime     = df['time'].max()
numCycles   = math.trunc((endTime - startTime) / (1/frequency))
df['peaks'] = 0.0

# Interate through all peaks to be highlighted
for peak in range(numCycles):
    # Generate the 'perfect' time that the peak is supposed to be
    currentTimePerfect = startTime + (peak * (1/frequency))

    # Find the 'closest' line in df to add the peak to
    for i in df.index:
        if df['time'][i] > currentTimePerfect:
            df['peaks'][i] = df['voltage'].max()
            break

# Iterate again through df and evaluate all datapoints where peaks is set
bitmem = []
skipper = False
df['bits'] = 0.0
for i in df.index:
    # Only test lines where peaks marker is set
    if df['peaks'][i] > 0:
        # Only test every other marker as bits are duplicated
        if not skipper:
            if df['voltageLatched'][i] > 0:
                bitmem.append(1)
                df['bits'][i] = 1
            elif df['voltageLatched'][i] < 0:
                bitmem.append(0)
                df['bits'][i] = -1
            else:
                bitmem.append(-1)
                df['bits'][i] = 0

        skipper = not skipper

# Split bitmem into byte-sized chunks
message = []
for i in range(0, len(bitmem), 8):
        bytemem = bitmem[i:i + 8]
        byte = 0
        # only assemble bytes that contain full 8 bits of valid data
        if not -1 in bytemem:
            for bit in bytemem:
                byte = (byte << 1) | bit
        print(bytemem)
        message.append(f"{byte:#0{4}x}")

print(message)

print(bitmem)

# Plot data
fig = px.line(df, x = 'time', y = ['voltage', 'voltageLatched', 'peaks', 'bits'], title=relPath, template='plotly_dark')
fig.show()

Peak found at: 1.37e-08 with a height of: 0.4040201
[1, 1, 1, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 1, 1]
[0, 1, 1, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1]
[1, 1, 0, 1, 0, 1, 1, 0]
[0, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, 0, 0, -1, 1]
[-1, 0, -1, 1, -1, -1, 1, -1]
[-1, 1, -1, 0, -1, 1, 1, 1]
[1, 1, 1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1]
['0xe6', '0x20', '0x53', '0x69', '0x02', '0x00', '0x0b', '0xd6', '0x00', '0x00', '0x00', '0x00', '0x00', '0x00']
[1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, 1, -1, 0, -1, 1, -1, -1, 1, -1, -1, 1, -1, 0, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
